In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
import turbo as tb
import turbo.modules as tm
import turbo.gui.jupyter as tg
import turbo.plotting as tp

In [ ]:
import pygame_task
from functions import *
from function_optimisation import *
import function_optimisation as fo

The function to be optimised here is the velocity for the ball as a function of the y position of the ball.

This is hopefully a more robust solution to using time since in the real world the policy may not be executed exactly the same each time

In [ ]:
sim = pygame_task.Simulation()

In [ ]:
xmin, xmax = 0, sim.h
fo.xmin, fo.xmax = xmin, xmax
#hmin, hmax = -100, 100  # velocity
hmin, hmax = 0, sim.w   # position

num_controls = 5
cs = evenly_spaced(xmin, xmax, num_controls)
bounds = [('h{:02d}'.format(i), hmin, hmax) for i in range(num_controls)]

def build_fun(args):
    args = parse_args(args)
    heights = args['h']
    assert len(cs) == len(heights)
    return Piecewise(list(zip(cs, heights)), interpolation='quadratic')
    
def objective(**args):
    f = build_fun(args)
    return sim.run(fps=2000, ball_control=f, quiet=True)

In [ ]:
recs = turbo_opt(objective, bounds, desired_extremum='max')
#recs = random_opt(objective, bounds, desired_extremum='max', max_trials=80)

In [ ]:
plot_summary(recs, None, build_fun)

In [ ]:
def show_traj(optimiser, trial):
    args = optimiser._point_to_dict(trial.x)
    print(parse_args(args))
    f = build_fun(args)
    sim.show_trail(f)
    
def show_inc_trajs(recs):
    for r in recs:
        inc = r.get_incumbent()
        print('trial {} cost {}'.format(inc[0], inc[1].y))
        show_traj(r.optimiser, inc[1])

In [ ]:
show_inc_trajs(recs)